In [1]:
import numpy as np
import pandas as pd
import os

In [ ]:
!sudo rm -r /kaggle/working/VOT_V1

In [2]:
!git clone https://github.com/SinghArindam/VOT_V1/

Cloning into 'VOT_V1'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 133 (delta 28), reused 131 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 2.35 MiB | 25.38 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [3]:
!pip install -r /kaggle/working/VOT_V1/requirements.txt

In [7]:
!python /kaggle/working/VOT_V1/generate_traintxt.py

In [8]:
!python /kaggle/working/VOT_V1/trainSpeakerNet.py --config /kaggle/working/VOT_V1/configs/VOT_focal.yaml

Python Version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
PyTorch Version: 2.5.1+cu121
Number of GPUs: 2
Save path: exps/VOT_focal
Initialised focal margin 0.300 scale 30.000
Traceback (most recent call last):
  File "/kaggle/working/VOT_V1/trainSpeakerNet.py", line 279, in <module>
    main()
  File "/kaggle/working/VOT_V1/trainSpeakerNet.py", line 275, in main
    main_worker(0, None, args)
  File "/kaggle/working/VOT_V1/trainSpeakerNet.py", line 144, in main_worker
    train_dataset = train_dataset_loader(**vars(args))
  File "/kaggle/working/VOT_V1/DatasetLoader.py", line 127, in __init__
    with open(train_list) as dataset_file:
FileNotFoundError: [Errno 2] No such file or directory: 'b.txt'


In [5]:
# !python ./trainSpeakerNet.py --eval --config ./configs/VOT_focal.yaml --initial_model exps/VOT_focal/model/model000000007.model

# Combined V1

In [6]:
# #!/usr/bin/env python
# # -*- coding: utf-8 -*-
# """
# Combined Voice-Transformer Speaker Verification Code for Kaggle

# This file is an all-in-one Python script that combines all of the repository’s code:
#   - utils.py
#   - DatasetLoader.py
#   - Feature_Visualization.py
#   - SpeakerNet.py
#   - dataprep.py
#   - generate_traintxt.py
#   - trainSpeakerNet.py
#   - tuneThreshold.py
# and also (inlined below) the code for submodules from:
#   - models/
#   - loss/
#   - optimizer/
#   - scheduler/

# Before running on Kaggle, please ensure that:
#   • All external dependencies (e.g. torch, torchaudio, numpy, scipy, pyyaml, tqdm, soundfile, scikit-learn) are installed.
#   • The [SUBMODULES] sections are filled in with the original code.
#   • Any file path (for datasets, configuration files, etc.) is modified appropriately for the Kaggle environment.

# ---------------------------------------------------------------------
# Author: (Your Name)
# Date: (Current Date)
# ---------------------------------------------------------------------
# """

# ############################################
# # --- Begin utils.py (31 lines) ---
# ############################################
# import torch
# import torch.nn.functional as F

# def accuracy(output, target, topk=(1,)):
#     """Computes the precision@k for the specified values of k"""
#     maxk = max(topk)
#     batch_size = target.size(0)
#     _, pred = output.topk(maxk, 1, True, True)
#     pred = pred.t()
#     correct = pred.eq(target.view(1, -1).expand_as(pred))
#     res = []
#     for k in topk:
#         correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
#         res.append(correct_k.mul_(100.0 / batch_size))
#     return res

# class PreEmphasis(torch.nn.Module):
#     def __init__(self, coef: float = 0.97):
#         super().__init__()
#         self.coef = coef
#         # In pytorch, convolution uses cross-correlation so we flip the filter.
#         self.register_buffer('flipped_filter', torch.FloatTensor([-self.coef, 1.]).unsqueeze(0).unsqueeze(0))
#     def forward(self, input: torch.tensor) -> torch.tensor:
#         assert len(input.size()) == 2, 'Input tensor must be 2D!'
#         input = input.unsqueeze(1)
#         input = F.pad(input, (1, 0), 'reflect')
#         return F.conv1d(input, self.flipped_filter).squeeze(1)
# ############################################
# # --- End utils.py ---
# ############################################

# ############################################
# # --- Begin DatasetLoader.py (from 305-line file) ---
# ############################################
# import os
# import random
# import threading
# import time
# import math
# import json
# import torchaudio
# import glob
# import soundfile
# from scipy import signal
# from scipy.io import wavfile
# from torch.utils.data import Dataset, DataLoader
# import torch.distributed as dist
# import numpy

# def round_down(num, divisor):
#     return num - (num % divisor)

# def worker_init_fn(worker_id):
#     numpy.random.seed(numpy.random.get_state()[1][0] + worker_id)

# def loadWAV(filename, max_frames, evalmode=True, num_eval=10):
#     # Maximum audio length
#     max_audio = max_frames * 160
#     # Read wav file and convert to numpy array
#     audio, sample_rate = soundfile.read(filename)
#     audiosize = audio.shape[0]
#     if audiosize <= max_audio:
#         shortage = max_audio - audiosize + 1
#         audio = numpy.pad(audio, (0, shortage), 'wrap')
#         audiosize = audio.shape[0]
#     if evalmode:
#         startframe = numpy.linspace(0, audiosize - max_audio, num=num_eval)
#     else:
#         startframe = numpy.array([numpy.int64(random.random() * (audiosize - max_audio))])
#     feats = []
#     if evalmode and max_frames == 0:
#         feats.append(audio)
#     else:
#         for asf in startframe:
#             x = audio[int(asf):int(asf) + max_audio]
#             feats.append(x)
#     feat = numpy.stack(feats, axis=0).astype(numpy.float)
#     return feat

# class AugmentWAV(object):
#     def __init__(self, musan_path, rir_path, max_frames):
#         self.max_frames = max_frames
#         self.max_audio = max_frames * 160 + 240
#         self.noisetypes = ['noise', 'speech', 'music']
#         self.noisesnr = {'noise': [0,15], 'speech': [13,20], 'music': [5,15]}
#         self.numnoise = {'noise': [1,1], 'speech': [3,7], 'music': [1,1]}
#         self.noiselist = {}
#         augment_files = glob.glob(os.path.join(musan_path, '*/*/*/*.wav'))
#         for file in augment_files:
#             key = file.split('/')[-4]
#             if key not in self.noiselist:
#                 self.noiselist[key] = []
#             self.noiselist[key].append(file)
#         self.rir_files = glob.glob(os.path.join(rir_path, '*/*/*.wav'))
#     def additive_noise(self, noisecat, audio):
#         clean_db = 10 * numpy.log10(numpy.mean(audio ** 2) + 1e-4)
#         numnoise = self.numnoise[noisecat]
#         noiselist = random.sample(self.noiselist[noisecat], random.randint(numnoise[0], numnoise[1]))
#         noises = []
#         for noise in noiselist:
#             noiseaudio = loadWAV(noise, self.max_frames, evalmode=False)
#             noise_snr = random.uniform(self.noisesnr[noisecat][0], self.noisesnr[noisecat][1])
#             noise_db = 10 * numpy.log10(numpy.mean(noiseaudio[0] ** 2) + 1e-4)
#             noises.append(numpy.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10)) * noiseaudio)
#         return numpy.sum(numpy.concatenate(noises, axis=0), axis=0, keepdims=True) + audio
#     def reverberate(self, audio):
#         rir_file = random.choice(self.rir_files)
#         rir, fs = soundfile.read(rir_file)
#         rir = numpy.expand_dims(rir.astype(numpy.float), 0)
#         rir = rir / numpy.sqrt(numpy.sum(rir ** 2))
#         return signal.convolve(audio, rir, mode='full')[:, :self.max_audio]

# class train_dataset_loader(Dataset):
#     def __init__(self, train_list, augment, musan_path, rir_path, max_frames, train_path, **kwargs):
#         self.augment_wav = AugmentWAV(musan_path=musan_path, rir_path=rir_path, max_frames=max_frames)
#         self.train_list = train_list
#         self.max_frames = max_frames
#         self.musan_path = musan_path
#         self.rir_path = rir_path
#         self.augment = augment
#         with open(train_list) as dataset_file:
#             lines = dataset_file.readlines()
#         dictkeys = list(set([x.split()[0] for x in lines]))
#         dictkeys.sort()
#         dictkeys = { key: ii for ii, key in enumerate(dictkeys) }
#         self.data_list = []
#         self.data_label = []
#         for lidx, line in enumerate(lines):
#             data = line.strip().split()
#             speaker_label = dictkeys[data[0]]
#             filename = os.path.join(train_path, data[1])
#             self.data_label.append(speaker_label)
#             self.data_list.append(filename)
#     def __getitem__(self, indices):
#         feat = []
#         for index in indices:
#             waveform, sample_rate = torchaudio.load(self.data_list[index])
#             feature = torchaudio.compliance.kaldi.fbank(
#                 waveform, num_mel_bins=111, dither=0, energy_floor=0,
#                 low_freq=20, high_freq=0, preemphasis_coefficient=0.97,
#                 use_energy=True, raw_energy=True, remove_dc_offset=True,
#                 sample_frequency=16000, window_type='povey')
#             xs = feature
#             xssize = xs.shape[0]
#             startframe = numpy.array([numpy.int64(random.random()*(xssize - self.max_frames))])
#             feats = []
#             if self.augment:
#                 augtype = random.randint(0, 4)
#                 # Uncomment and adjust the following lines if you want to augment the audio:
#                 # if augtype == 1:
#                 #     audio = self.augment_wav.reverberate(audio)
#                 # elif augtype == 2:
#                 #     audio = self.augment_wav.additive_noise('music', audio)
#                 # elif augtype == 3:
#                 #     audio = self.augment_wav.additive_noise('speech', audio)
#                 # elif augtype == 4:
#                 #     audio = self.augment_wav.additive_noise('noise', audio)
#                 pass
#             for asf in startframe:
#                 x = xs[int(asf):int(asf) + self.max_frames]
#                 feats.append(x)
#             feats = numpy.stack(feats, axis=0).astype(numpy.float)
#             feat.append(feats)
#         feat = numpy.concatenate(feat, axis=0)
#         return torch.FloatTensor(feat), self.data_label[index]
#     def __len__(self):
#         return len(self.data_list)

# class test_dataset_loader(Dataset):
#     def __init__(self, test_list, test_path, eval_frames, num_eval, **kwargs):
#         self.max_frames = eval_frames
#         self.num_eval = num_eval
#         self.test_path = test_path
#         self.test_list = test_list
#     def __getitem__(self, index):
#         waveform, sample_rate = torchaudio.load(os.path.join(self.test_path, self.test_list[index]))
#         feature = torchaudio.compliance.kaldi.fbank(
#             waveform, num_mel_bins=111, dither=0, energy_floor=0,
#             low_freq=20, high_freq=0, preemphasis_coefficient=0.97,
#             use_energy=True, raw_energy=True, remove_dc_offset=True,
#             sample_frequency=16000, window_type='povey')
#         xs = feature
#         xssize = xs.shape[0]
#         feats = []
#         startframe = numpy.linspace(0, xssize - self.max_frames, num=10)
#         for asf in startframe:
#             x = xs[int(asf):int(asf) + self.max_frames]
#             feats.append(x)
#         feat = numpy.stack(feats, axis=0).astype(numpy.float)
#         return torch.FloatTensor(feat), self.test_list[index]
#     def __len__(self):
#         return len(self.test_list)

# class train_dataset_sampler(torch.utils.data.Sampler):
#     def __init__(self, data_source, nPerSpeaker, max_seg_per_spk, batch_size, distributed, seed, **kwargs):
#         self.data_label = data_source.data_label
#         self.nPerSpeaker = nPerSpeaker
#         self.max_seg_per_spk = max_seg_per_spk
#         self.batch_size = batch_size
#         self.epoch = 0
#         self.seed = seed
#         self.distributed = distributed
#     def __iter__(self):
#         g = torch.Generator()
#         g.manual_seed(self.seed + self.epoch)
#         indices = torch.randperm(len(self.data_label), generator=g).tolist()
#         data_dict = {}
#         for index in indices:
#             speaker_label = self.data_label[index]
#             if speaker_label not in data_dict:
#                 data_dict[speaker_label] = []
#             data_dict[speaker_label].append(index)
#         dictkeys = list(data_dict.keys())
#         dictkeys.sort()
#         lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
#         flattened_list = []
#         flattened_label = []
#         for findex, key in enumerate(dictkeys):
#             data = data_dict[key]
#             numSeg = round_down(min(len(data), self.max_seg_per_spk), self.nPerSpeaker)
#             rp = lol(numpy.arange(numSeg), self.nPerSpeaker)
#             flattened_label.extend([findex] * (len(rp)))
#             for indices in rp:
#                 flattened_list.append([data[i] for i in indices])
#         mixid = torch.randperm(len(flattened_label), generator=g).tolist()
#         mixlabel = []
#         mixmap = []
#         for ii in mixid:
#             startbatch = round_down(len(mixlabel), self.batch_size)
#             if flattened_label[ii] not in mixlabel[startbatch:]:
#                 mixlabel.append(flattened_label[ii])
#                 mixmap.append(ii)
#         mixed_list = [flattened_list[i] for i in mixmap]
#         if self.distributed:
#             total_size = round_down(len(mixed_list), self.batch_size * dist.get_world_size())
#             start_index = int((dist.get_rank()) / dist.get_world_size() * total_size)
#             end_index = int((dist.get_rank() + 1) / dist.get_world_size() * total_size)
#             self.num_samples = end_index - start_index
#             return iter(mixed_list[start_index:end_index])
#         else:
#             total_size = round_down(len(mixed_list), self.batch_size)
#             self.num_samples = total_size
#             return iter(mixed_list[:total_size])
#     def __len__(self) -> int:
#         return self.num_samples
#     def set_epoch(self, epoch: int) -> None:
#         self.epoch = epoch
# ############################################
# # --- End DatasetLoader.py ---
# ############################################

# ############################################
# # --- Begin Feature_Visualization.py (42 lines) ---
# ############################################
# import matplotlib.pyplot as plt
# import os
# import torchaudio

# # Set these paths appropriately in Kaggle.
# output_images_folder = "output_images"  # Directory to save images
# data_path_list = "input_wav_files"       # Directory to input .wav files

# for root, dirs, files in os.walk(data_path_list):
#     rel_path = os.path.relpath(root, data_path_list)
#     output_folder = os.path.join(output_images_folder, rel_path)
#     for wav_file in files:
#         if wav_file.endswith('.wav'):
#             wav_path = os.path.join(root, wav_file)
#             print("Processing:", wav_path)
#             waveform, sample_rate = torchaudio.load(wav_path)
#             feature = torchaudio.compliance.kaldi.fbank(
#                 waveform, num_mel_bins=111, dither=0, energy_floor=0,
#                 low_freq=20, high_freq=0, preemphasis_coefficient=0.97,
#                 use_energy=True, raw_energy=True, remove_dc_offset=True,
#                 sample_frequency=16000, window_type='povey')
#             xs = feature
#             plt.imshow(xs.detach().numpy(), cmap='viridis', origin='lower', aspect='auto')
#             plt.axis("off")
#             output_filename = os.path.splitext(os.path.basename(wav_path))[0] + '.png'
#             os.makedirs(output_folder, exist_ok=True)
#             output_path = os.path.join(output_folder, output_filename)
#             plt.savefig(output_path, bbox_inches="tight", pad_inches=0)
#             plt.show()
# ############################################
# # --- End Feature_Visualization.py ---
# ############################################

# ############################################
# # --- Begin SpeakerNet.py (282 lines) ---
# ############################################
# import importlib
# import torch.nn as nn
# import torch.nn.functional as F
# import sys
# import time
# import itertools
# from torch.cuda.amp import autocast, GradScaler
# # Note: In the original code, there is a dependency: "from DatasetLoader import test_dataset_loader"
# # (which is already defined above).

# class WrappedModel(nn.Module):
#     """A wrapper to enable consistency between single- and multi-GPU training."""
#     def __init__(self, model):
#         super(WrappedModel, self).__init__()
#         self.module = model
#     def forward(self, x, label=None):
#         return self.module(x, label)

# class SpeakerNet(nn.Module):
#     def __init__(self, model, optimizer, trainfunc, nPerSpeaker, **kwargs):
#         super(SpeakerNet, self).__init__()
#         # Dynamically import the model from the 'models' folder.
#         # [SUBMODULE: Paste the code from your models module here, e.g. MainModel definition]
#         SpeakerNetModel = importlib.import_module("models." + model).__getattribute__("MainModel")
#         self.__S__ = SpeakerNetModel(**kwargs)
#         # Dynamically import the loss function from the 'loss' folder.
#         # [SUBMODULE: Paste the code from your loss module here (e.g. AAMF loss or other)]
#         LossFunction = importlib.import_module("loss." + trainfunc).__getattribute__("LossFunction")
#         self.__L__ = LossFunction(**kwargs)
#         self.nPerSpeaker = nPerSpeaker
#     def forward(self, data, label=None):
#         outp = self.__S__.forward(data)
#         if label is None:
#             return outp
#         else:
#             nloss, prec1 = self.__L__.forward(outp, label)
#             return nloss, prec1, outp

# class ModelTrainer(object):
#     def __init__(self, speaker_model, optimizer, scheduler, gpu, mixedprec, **kwargs):
#         self.__model__ = speaker_model
#         # Dynamically import the optimizer from the 'optimizer' folder.
#         # [SUBMODULE: Paste the code from your optimizer module here]
#         Optimizer = importlib.import_module("optimizer." + optimizer).__getattribute__("Optimizer")
#         self.__optimizer__ = Optimizer(self.__model__.parameters(), **kwargs)
#         # Dynamically import the scheduler from the 'scheduler' folder.
#         # [SUBMODULE: Paste the code from your scheduler module here]
#         Scheduler = importlib.import_module("scheduler." + scheduler).__getattribute__("Scheduler")
#         self.__scheduler__, self.lr_step = Scheduler(self.__optimizer__, **kwargs)
#         self.scaler = GradScaler()
#         self.gpu = gpu
#         self.mixedprec = mixedprec
#         assert self.lr_step in ["epoch", "iteration"]
#     def train_network(self, loader, verbose):
#         self.__model__.train()
#         stepsize = loader.batch_size
#         counter = 0
#         index = 0
#         loss = 0
#         top1 = 0
#         tstart = time.time()
#         a = [[]]
#         for data, data_label in loader:
#             self.__model__.zero_grad()
#             label = torch.LongTensor(data_label).cuda()
#             nloss, prec1, outp = self.__model__(data, label)
#             a.append(outp.tolist())
#             nloss.backward()
#             self.__optimizer__.step()
#             loss += nloss.detach().cpu().item()
#             top1 += prec1.detach().cpu().item()
#             counter += 1
#             index += stepsize
#             telapsed = time.time() - tstart
#             tstart = time.time()
#             if verbose:
#                 sys.stdout.write("\rProcessing {:d} of {:d}:".format(index, loader.__len__() * loader.batch_size))
#                 sys.stdout.write("Loss {:f} TEER/TAcc {:2.3f}% - {:.2f} Hz ".format(loss / counter, top1 / counter, stepsize / telapsed))
#                 sys.stdout.flush()
#             if self.lr_step == "iteration":
#                 self.__scheduler__.step()
#         if self.lr_step == "epoch":
#             self.__scheduler__.step()
#         return (loss / counter, top1 / counter)
#     def evaluateFromList(self, test_list, test_path, nDataLoaderThread, distributed, print_interval=100, num_eval=10, **kwargs):
#         if distributed:
#             rank = torch.distributed.get_rank()
#         else:
#             rank = 0
#         self.__model__.eval()
#         with open(test_list) as f:
#             lines = f.readlines()
#         files = list(set(list(itertools.chain(*[x.strip().split()[-2:] for x in lines]))))
#         files.sort()
#         feats = {}
#         tstart = time.time()
#         from DatasetLoader import test_dataset_loader  # Already defined above.
#         test_dataset = test_dataset_loader(files, test_path, num_eval=num_eval, **kwargs)
#         if distributed:
#             sampler = torch.utils.data.distributed.DistributedSampler(test_dataset, shuffle=False)
#         else:
#             sampler = None
#         test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=nDataLoaderThread, drop_last=False, sampler=sampler)
#         for idx, data in enumerate(test_loader):
#             inp1 = data[0][0]
#             with torch.no_grad():
#                 ref_feat = self.__model__(inp1).detach().cpu()
#             feats[data[1][0]] = ref_feat
#             telapsed = time.time() - tstart
#             if idx % print_interval == 0 and rank == 0:
#                 sys.stdout.write("\rReading {:d} of {:d}: {:.2f} Hz, embedding size {:d}".format(idx, test_loader.__len__(), idx / telapsed, ref_feat.size()[1]))
#         all_scores = []
#         all_labels = []
#         all_trials = []
#         if distributed:
#             feats_all = [None for _ in range(torch.distributed.get_world_size())]
#             torch.distributed.all_gather_object(feats_all, feats)
#             if rank == 0:
#                 tstart = time.time()
#                 feats = feats_all[0]
#                 for feats_batch in feats_all[1:]:
#                     feats.update(feats_batch)
#         if rank == 0:
#             for idx, line in enumerate(lines):
#                 data = line.split()
#                 if len(data) == 2:
#                     data = [random.randint(0, 1)] + data
#                 ref_feat = feats[data[1]].cuda()
#                 com_feat = feats[data[2]].cuda()
#                 if self.__model__.module.__L__.test_normalize:
#                     ref_feat = F.normalize(ref_feat, p=2, dim=1)
#                     com_feat = F.normalize(com_feat, p=2, dim=1)
#                 dist_mat = torch.cdist(ref_feat.reshape(num_eval, -1), com_feat.reshape(num_eval, -1)).detach().cpu().numpy()
#                 score = -1 * numpy.mean(dist_mat)
#                 all_scores.append(score)
#                 all_labels.append(int(data[0]))
#                 all_trials.append(data[1] + " " + data[2])
#                 if idx % print_interval == 0:
#                     telapsed = time.time() - tstart
#                     sys.stdout.write("\rComputing {:d} of {:d}: {:.2f} Hz".format(idx, len(lines), idx / telapsed))
#                     sys.stdout.flush()
#         return (all_scores, all_labels, all_trials)
#     def saveParameters(self, path):
#         torch.save(self.__model__.module.state_dict(), path)
#     def loadParameters(self, path):
#         self_state = self.__model__.module.state_dict()
#         loaded_state = torch.load(path, map_location="cuda:%d" % self.gpu)
#         if len(loaded_state.keys()) == 1 and "model" in loaded_state:
#             loaded_state = loaded_state["model"]
#         newdict = {}
#         delete_list = []
#         for name, param in loaded_state.items():
#             new_name = "__S__." + name
#             newdict[new_name] = param
#             delete_list.append(name)
#         loaded_state.update(newdict)
#         for name in delete_list:
#             del loaded_state[name]
#         for name, param in loaded_state.items():
#             origname = name
#             if name not in self_state:
#                 name = name.replace("module.", "")
#             if name not in self_state:
#                 print("{} is not in the model.".format(origname))
#                 continue
#             if self_state[name].size() != loaded_state[origname].size():
#                 print("Wrong parameter length: {}, model: {}, loaded: {}".format(origname, self_state[name].size(), loaded_state[origname].size()))
#                 continue
#             self_state[name].copy_(param)
# ############################################
# # --- End SpeakerNet.py ---
# ############################################

# ############################################
# # --- Begin dataprep.py (206 lines) ---
# ############################################
# import argparse
# import subprocess
# import hashlib
# import tarfile
# from zipfile import ZipFile
# from tqdm import tqdm
# from scipy.io import wavfile

# parser_dataprep = argparse.ArgumentParser(description="VoxCeleb downloader")
# parser_dataprep.add_argument('--save_path', type=str, default="data", help='Target directory')
# parser_dataprep.add_argument('--user', type=str, default="user", help='Username')
# parser_dataprep.add_argument('--password', type=str, default="pass", help='Password')
# parser_dataprep.add_argument('--download', dest='download', action='store_true', help='Enable download')
# parser_dataprep.add_argument('--extract', dest='extract', action='store_true', help='Enable extract')
# parser_dataprep.add_argument('--convert', dest='convert', action='store_true', help='Enable convert')
# parser_dataprep.add_argument('--augment', dest='augment', action='store_true', help='Download and extract augmentation files')
# args_dataprep = parser_dataprep.parse_args()

# def md5(fname):
#     hash_md5 = hashlib.md5()
#     with open(fname, "rb") as f:
#         for chunk in iter(lambda: f.read(4096), b""):
#             hash_md5.update(chunk)
#     return hash_md5.hexdigest()

# def download_files(args, lines):
#     for line in lines:
#         url = line.split()[0]
#         md5gt = line.split()[1]
#         outfile = url.split('/')[-1]
#         out = subprocess.call('wget %s --user %s --password %s -O %s/%s' % (url, args.user, args.password, args.save_path, outfile), shell=True)
#         if out != 0:
#             raise ValueError('Download failed %s.' % url)
#         md5ck = md5('%s/%s' % (args.save_path, outfile))
#         if md5ck == md5gt:
#             print('Checksum successful %s.' % outfile)
#         else:
#             raise Warning('Checksum failed %s.' % outfile)

# def concatenate_files(args, lines):
#     for line in lines:
#         infile = line.split()[0]
#         outfile = line.split()[1]
#         md5gt = line.split()[2]
#         out = subprocess.call('cat %s/%s > %s/%s' % (args.save_path, infile, args.save_path, outfile), shell=True)
#         md5ck = md5('%s/%s' % (args.save_path, outfile))
#         if md5ck == md5gt:
#             print('Checksum successful %s.' % outfile)
#         else:
#             raise Warning('Checksum failed %s.' % outfile)
#         subprocess.call('rm %s/%s' % (args.save_path, infile), shell=True)

# def safe_extract(tar, path=".", members=None, *, numeric_owner=False):
#     import os
#     def is_within_directory(directory, target):
#         abs_directory = os.path.abspath(directory)
#         abs_target = os.path.abspath(target)
#         prefix = os.path.commonprefix([abs_directory, abs_target])
#         return prefix == abs_directory
#     for member in tar.getmembers():
#         member_path = os.path.join(path, member.name)
#         if not is_within_directory(path, member_path):
#             raise Exception("Attempted Path Traversal in Tar File")
#     tar.extractall(path, members, numeric_owner=numeric_owner)

# def full_extract(args, fname):
#     print('Extracting %s' % fname)
#     if fname.endswith(".tar.gz"):
#         with tarfile.open(fname, "r:gz") as tar:
#             safe_extract(tar, args.save_path)
#     elif fname.endswith(".zip"):
#         with ZipFile(fname, 'r') as zf:
#             zf.extractall(args.save_path)

# def part_extract(args, fname, target):
#     print('Extracting %s' % fname)
#     with ZipFile(fname, 'r') as zf:
#         for infile in zf.namelist():
#             if any([infile.startswith(x) for x in target]):
#                 zf.extract(infile, args.save_path)

# def convert_aac_to_wav(args):
#     import glob
#     files = glob.glob('%s/voxceleb2/*/*/*.m4a' % args.save_path)
#     files.sort()
#     print('Converting files from AAC to WAV')
#     for fname in tqdm(files):
#         outfile = fname.replace('.m4a', '.wav')
#         out = subprocess.call('ffmpeg -y -i %s -ac 1 -vn -acodec pcm_s16le -ar 16000 %s >/dev/null 2>/dev/null' % (fname, outfile), shell=True)
#         if out != 0:
#             raise ValueError('Conversion failed %s.' % fname)

# def split_musan(args):
#     import glob
#     files = glob.glob('%s/musan/*/*/*.wav' % args.save_path)
#     audlen = 16000 * 5
#     audstr = 16000 * 3
#     for idx, file in enumerate(files):
#         fs, aud = wavfile.read(file)
#         writedir = os.path.splitext(file.replace('/musan/', '/musan_split/'))[0]
#         os.makedirs(writedir, exist_ok=True)
#         for st in range(0, len(aud) - audlen, audstr):
#             wavfile.write(writedir + '/%05d.wav' % (st/fs), fs, aud[st:st+audlen])
#         print(idx, file)

# if __name__ == "__main__" and False:
#     # This block is only run if you call dataprep.py directly.
#     if not os.path.exists(args_dataprep.save_path):
#         raise ValueError('Target directory does not exist.')
#     with open('lists/fileparts.txt', 'r') as f:
#         fileparts = f.readlines()
#     with open('lists/files.txt', 'r') as f:
#         files = f.readlines()
#     with open('lists/augment.txt', 'r') as f:
#         augfiles = f.readlines()
#     if args_dataprep.augment:
#         download_files(args_dataprep, augfiles)
#         part_extract(args_dataprep, os.path.join(args_dataprep.save_path, 'rirs_noises.zip'),
#                      ['RIRS_NOISES/simulated_rirs/mediumroom', 'RIRS_NOISES/simulated_rirs/smallroom'])
#         full_extract(args_dataprep, os.path.join(args_dataprep.save_path, 'musan.tar.gz'))
#         split_musan(args_dataprep)
#     if args_dataprep.download:
#         download_files(args_dataprep, fileparts)
#     if args_dataprep.extract:
#         concatenate_files(args_dataprep, files)
#         for file in files:
#             full_extract(args_dataprep, os.path.join(args_dataprep.save_path, file.split()[1]))
#         subprocess.call('mv %s/dev/aac/* %s/aac/ && rm -r %s/dev' % (args_dataprep.save_path, args_dataprep.save_path, args_dataprep.save_path), shell=True)
#         subprocess.call('mv %s/wav %s/voxceleb1' % (args_dataprep.save_path, args_dataprep.save_path), shell=True)
#         subprocess.call('mv %s/aac %s/voxceleb2' % (args_dataprep.save_path, args_dataprep.save_path), shell=True)
#     if args_dataprep.convert:
#         convert_aac_to_wav(args_dataprep)
# ############################################
# # --- End dataprep.py ---
# ############################################

# ############################################
# # --- Begin generate_traintxt.py (6 lines) ---
# ############################################
# import os
# for dirpath, dirnames, filenames in os.walk('path/data/'):
#     a = dirpath.split('data/')[-1]
#     for filename in filenames:
#         print(a + " " + dirpath.split('data/')[-1] + "/" + filename)
# ############################################
# # --- End generate_traintxt.py ---
# ############################################

# ############################################
# # --- Begin trainSpeakerNet.py (279 lines) ---
# ############################################
# import argparse
# import yaml
# import glob
# import zipfile
# import datetime
# import torch
# import torch.distributed as dist
# import torch.multiprocessing as mp
# import warnings
# import sys
# import random

# # Import the tuneThreshold functions (defined later in this file)
# # (They are already included below in tuneThreshold.py section.)
# # from tuneThreshold import *

# # Import SpeakerNet and DatasetLoader (already defined above)
# # from SpeakerNet import SpeakerNet, WrappedModel, ModelTrainer
# # from DatasetLoader import train_dataset_loader, train_dataset_sampler, worker_init_fn

# parser = argparse.ArgumentParser(description="SpeakerNet")
# parser.add_argument('--config', type=str, default=None, help='Config YAML file')
# parser.add_argument('--max_frames', type=int, default=200, help='Input length for training')
# parser.add_argument('--eval_frames', type=int, default=300, help='Input length for testing; 0 uses whole file')
# parser.add_argument('--batch_size', type=int, default=200, help='Batch size (number of speakers per batch)')
# parser.add_argument('--max_seg_per_spk', type=int, default=500, help='Max utterances per speaker per epoch')
# parser.add_argument('--nDataLoaderThread', type=int, default=24, help='Number of data loader threads')
# parser.add_argument('--augment', type=bool, default=False, help='Augment input')
# parser.add_argument('--seed', type=int, default=3407, help='Random seed')
# parser.add_argument('--test_interval', type=int, default=2, help='Test and save every [test_interval] epochs')
# parser.add_argument('--max_epoch', type=int, default=500, help='Maximum number of epochs')
# parser.add_argument('--trainfunc', type=str, default="", help='Loss function')
# parser.add_argument('--optimizer', type=str, default="adam", help='Optimizer: sgd or adam')
# parser.add_argument('--scheduler', type=str, default="steplr", help='Learning rate scheduler')
# parser.add_argument('--lr', type=float, default=0.0005, help='Learning rate')
# parser.add_argument("--lr_decay", type=float, default=0.95, help='Learning rate decay every test_interval epochs')
# parser.add_argument('--weight_decay', type=float, default=0.1, help='Weight decay')
# parser.add_argument("--hard_prob", type=float, default=0.5, help='Hard negative mining probability')
# parser.add_argument("--hard_rank", type=int, default=10, help='Hard negative mining rank in the batch')
# parser.add_argument('--margin', type=float, default=0.3, help='Loss margin')
# parser.add_argument('--scale', type=float, default=30, help='Loss scale')
# parser.add_argument('--nPerSpeaker', type=int, default=1, help='Utterances per speaker per batch')
# parser.add_argument('--nClasses', type=int, default=1211, help='Number of speakers in softmax layer')
# parser.add_argument('--dcf_p_target', type=float, default=0.05, help='A priori probability of target speaker')
# parser.add_argument('--dcf_c_miss', type=float, default=1, help='Cost of missed detection')
# parser.add_argument('--dcf_c_fa', type=float, default=1, help='Cost of false alarm')
# parser.add_argument('--initial_model', type=str, default="", help='Initial model weights')
# parser.add_argument('--save_path', type=str, default="exps/exp1", help='Path for models and logs')
# parser.add_argument('--train_list', type=str, default="b.txt", help='Training list')
# parser.add_argument('--test_list', type=str, default="veri_test1.txt", help='Evaluation list')
# parser.add_argument('--train_path', type=str, default="path1", help='Path to train set')
# parser.add_argument('--test_path', type=str, default="path2", help='Path to test set')
# parser.add_argument('--musan_path', type=str, default="data/musan_split", help='Path to MUSAN data')
# parser.add_argument('--rir_path', type=str, default="data/RIRS_NOISES/simulated_rirs", help='Path to RIRs')
# parser.add_argument('--n_mels', type=int, default=40, help='Number of mel filterbanks')
# parser.add_argument('--log_input', type=bool, default=False, help='Log input features')
# parser.add_argument('--model', type=str, default="", help='Name of model definition')
# parser.add_argument('--encoder_type', type=str, default="SAP", help='Type of encoder')
# parser.add_argument('--nOut', type=int, default=512, help='Embedding size in final FC layer')
# parser.add_argument('--sinc_stride', type=int, default=10, help='Stride size for RawNet3 analytic filterbank')
# parser.add_argument('--eval', dest='eval', action='store_true', help='Evaluation only')
# parser.add_argument('--port', type=str, default="8888", help='Port for distributed training')
# parser.add_argument('--distributed', dest='distributed', action='store_true', help='Enable distributed training')
# parser.add_argument('--mixedprec', dest='mixedprec', action='store_true', help='Enable mixed precision training')
# args_train = parser.parse_args()

# def find_option_type(key, parser):
#     for opt in parser._get_optional_actions():
#         if ('--' + key) in opt.option_strings:
#             return opt.type
#     raise ValueError

# if args_train.config is not None:
#     with open(args_train.config, "r") as f:
#         yml_config = yaml.load(f, Loader=yaml.FullLoader)
#     for k, v in yml_config.items():
#         if k in args_train.__dict__:
#             typ = find_option_type(k, parser)
#             args_train.__dict__[k] = typ(v)
#         else:
#             sys.stderr.write("Ignored unknown parameter {} in yaml.\n".format(k))

# def main_worker(gpu, ngpus_per_node, args):
#     args.gpu = gpu
#     # Initialize model
#     s = SpeakerNet(**vars(args))
#     if args.distributed:
#         os.environ['MASTER_ADDR'] = 'localhost'
#         os.environ['MASTER_PORT'] = args.port
#         dist.init_process_group(backend='nccl', world_size=ngpus_per_node, rank=args.gpu)
#         torch.cuda.set_device(args.gpu)
#         s.cuda(args.gpu)
#         s = torch.nn.parallel.DistributedDataParallel(s, device_ids=[args.gpu], find_unused_parameters=True)
#         print('Loaded model on GPU {:d}'.format(args.gpu))
#     else:
#         s = WrappedModel(s).cuda(args.gpu)
#     it = 1
#     eers = [100]
#     if args.gpu == 0:
#         scorefile = open(args.result_save_path + "/scores.txt", "a+")
#         from DatasetLoader import train_dataset_loader, train_dataset_sampler, worker_init_fn
#         train_dataset = train_dataset_loader(**vars(args))
#         train_sampler = train_dataset_sampler(train_dataset, **vars(args))
#         train_loader = torch.utils.data.DataLoader(
#             train_dataset,
#             batch_size=args.batch_size,
#             num_workers=args.nDataLoaderThread,
#             sampler=train_sampler,
#             pin_memory=False,
#             worker_init_fn=worker_init_fn,
#             drop_last=True,
#         )
#         trainer = ModelTrainer(s, **vars(args))
#         modelfiles = glob.glob('%s/model0*.model' % args.model_save_path)
#         modelfiles.sort()
#         if args.initial_model != "":
#             trainer.loadParameters(args.initial_model)
#             print("Model {} loaded!".format(args.initial_model))
#         elif len(modelfiles) >= 1:
#             trainer.loadParameters(modelfiles[-1])
#             print("Model {} loaded from previous state!".format(modelfiles[-1]))
#             it = int(os.path.splitext(os.path.basename(modelfiles[-1]))[0][5:]) + 1
#             for ii in range(1, it):
#                 trainer.__scheduler__.step()
#         if args.eval == True:
#             pytorch_total_params = sum(p.numel() for p in s.module.__S__.parameters())
#             print('Total parameters: ', pytorch_total_params)
#             print('Test list', args.test_list)
#             sc, lab, _ = trainer.evaluateFromList(**vars(args))
#             if args.gpu == 0:
#                 from tuneThreshold import tuneThresholdfromScore, ComputeErrorRates, ComputeMinDcf
#                 result = tuneThresholdfromScore(sc, lab, [1, 0.1])
#                 fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
#                 mindcf, threshold = ComputeMinDcf(fnrs, fprs, thresholds, args.dcf_p_target, args.dcf_c_miss, args.dcf_c_fa)
#                 sc = (numpy.array(sc) > threshold)
#                 test = (sc == numpy.array(lab))
#                 print('\n', time.strftime("%Y-%m-%d %H:%M:%S"), "VEER {:2.4f}".format(result[1]), "MinDCF {:2.5f}".format(mindcf))
#             return
#         if args.gpu == 0:
#             pyfiles = glob.glob('./*.py')
#             strtime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#             zipf = zipfile.ZipFile(args.result_save_path + '/run%s.zip' % strtime, 'w', zipfile.ZIP_DEFLATED)
#             for file in pyfiles:
#                 zipf.write(file)
#             zipf.close()
#             with open(args.result_save_path + '/run%s.cmd' % strtime, 'w') as f:
#                 f.write('%s' % args)
#         for it in range(it, args.max_epoch + 1):
#             train_sampler.set_epoch(it)
#             clr = [x['lr'] for x in trainer.__optimizer__.param_groups]
#             loss, traineer = trainer.train_network(train_loader, verbose=(args.gpu == 0))
#             if args.gpu == 0:
#                 print('\n', time.strftime("%Y-%m-%d %H:%M:%S"), "Epoch {:d}, TEER/TAcc {:2.2f}, TLOSS {:f}, LR {:f}".format(it, traineer, loss, max(clr)))
#                 scorefile.write("Epoch {:d}, TEER/TAcc {:2.2f}, TLOSS {:f}, LR {:f} \n".format(it, traineer, loss, max(clr)))
#             if it % 1 == 0:
#                 sc, lab, _ = trainer.evaluateFromList(**vars(args))
#                 if args.gpu == 0:
#                     from tuneThreshold import tuneThresholdfromScore, ComputeErrorRates, ComputeMinDcf
#                     result = tuneThresholdfromScore(sc, lab, [1, 0.1])
#                     fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
#                     mindcf, threshold = ComputeMinDcf(fnrs, fprs, thresholds, args.dcf_p_target, args.dcf_c_miss, args.dcf_c_fa)
#                     eers.append(result[1])
#                     print('\n', time.strftime("%Y-%m-%d %H:%M:%S"), "Epoch {:d}, VEER {:2.4f}, MinDCF {:2.5f}".format(it, result[1], mindcf))
#                     scorefile.write("Epoch {:d}, VEER {:2.4f}, MinDCF {:2.5f}\n".format(it, result[1], mindcf))
#                     trainer.saveParameters(args.model_save_path + "/model%09d.model" % it)
#                     with open(args.model_save_path + "/model%09d.eer" % it, 'w') as eerfile:
#                         eerfile.write('{:2.4f}'.format(result[1]))
#                     scorefile.flush()
#         if args.gpu == 0:
#             scorefile.close()

# def main():
#     args.model_save_path = args.save_path + "/model"
#     args.result_save_path = args.save_path + "/result"
#     args.feat_save_path = ""
#     os.makedirs(args.model_save_path, exist_ok=True)
#     os.makedirs(args.result_save_path, exist_ok=True)
#     n_gpus = torch.cuda.device_count()
#     print('Python Version:', sys.version)
#     print('PyTorch Version:', torch.__version__)
#     print('Number of GPUs:', n_gpus)
#     print('Save path:', args.save_path)
#     if args.distributed:
#         mp.spawn(main_worker, nprocs=n_gpus, args=(n_gpus, args))
#     else:
#         main_worker(0, None, args)

# if __name__ == '__main__':
#     main()
# ############################################
# # --- End trainSpeakerNet.py ---
# ############################################

# ############################################
# # --- Begin tuneThreshold.py (87 lines) ---
# ############################################
# from sklearn import metrics
# import numpy
# from operator import itemgetter

# def tuneThresholdfromScore(scores, labels, target_fa, target_fr=None):
#     fpr, tpr, thresholds = metrics.roc_curve(labels, scores, pos_label=1)
#     fnr = 1 - tpr
#     tunedThreshold = []
#     if target_fr:
#         for tfr in target_fr:
#             idx = numpy.nanargmin(numpy.absolute((tfr - fnr)))
#             tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]])
#     for tfa in target_fa:
#         idx = numpy.nanargmin(numpy.absolute((tfa - fpr)))
#         tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]])
#     idxE = numpy.nanargmin(numpy.absolute((fnr - fpr)))
#     eer = max(fpr[idxE], fnr[idxE]) * 100
#     return (tunedThreshold, eer, fpr, fnr)

# def ComputeErrorRates(scores, labels):
#     sorted_indexes, thresholds = zip(*sorted([(index, threshold) for index, threshold in enumerate(scores)], key=itemgetter(1)))
#     sorted_labels = [labels[i] for i in sorted_indexes]
#     fnrs = []
#     fprs = []
#     for i in range(0, len(sorted_labels)):
#         if i == 0:
#             fnrs.append(sorted_labels[i])
#             fprs.append(1 - sorted_labels[i])
#         else:
#             fnrs.append(fnrs[i-1] + sorted_labels[i])
#             fprs.append(fprs[i-1] + 1 - sorted_labels[i])
#     fnrs_norm = sum(sorted_labels)
#     fprs_norm = len(sorted_labels) - fnrs_norm
#     fnrs = [x / float(fnrs_norm) for x in fnrs]
#     fprs = [1 - x / float(fprs_norm) for x in fprs]
#     return fnrs, fprs, thresholds

# def ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa):
#     min_c_det = float("inf")
#     min_c_det_threshold = thresholds[0]
#     for i in range(0, len(fnrs)):
#         c_det = c_miss * fnrs[i] * p_target + c_fa * fprs[i] * (1 - p_target)
#         if c_det < min_c_det:
#             min_c_det = c_det
#             min_c_det_threshold = thresholds[i]
#     c_def = min(c_miss * p_target, c_fa * (1 - p_target))
#     min_dcf = min_c_det / c_def
#     return min_dcf, min_c_det_threshold
# ############################################
# # --- End tuneThreshold.py ---
# ############################################

# ############################################
# # [SUBMODULES]
# # --- Begin models modules ---
# ############################################
# # Paste here the code for all modules originally in the "models/" folder.
# # For example, if there is a file models/model_sym.py containing:
# #
# # class MainModel(nn.Module):
# #     def __init__(self, **kwargs):
# #         super(MainModel, self).__init__()
# #         # ... model layers ...
# #     def forward(self, x):
# #         # ... forward pass ...
# #         return x
# #
# # Then include its full source code here.
# ############################################
# # --- End models modules ---

# ############################################
# # --- Begin loss modules ---
# ############################################
# # Paste here the code for all modules originally in the "loss/" folder.
# ############################################
# # --- End loss modules ---

# ############################################
# # --- Begin optimizer modules ---
# ############################################
# # Paste here the code for all modules originally in the "optimizer/" folder.
# ############################################
# # --- End optimizer modules ---

# ############################################
# # --- Begin scheduler modules ---
# ############################################
# # Paste here the code for all modules originally in the "scheduler/" folder.
# ############################################
# # --- End scheduler modules ---


usage: colab_kernel_launcher.py [-h] [--save_path SAVE_PATH] [--user USER] [--password PASSWORD]
                                [--download] [--extract] [--convert] [--augment]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-3c5b317e-4171-486a-aca8-02c5fd460888.json


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Combined V2

In [ ]:
# #!/usr/bin/env python
# # -*- coding: utf-8 -*-
# """
# Optimized Combined Voice-Transformer Speaker Verification Code for Kaggle (Python 3.10)

# Key optimizations:
#   - Modern type hints and union type syntax (e.g. int | float)
#   - Mixed-precision training via torch.cuda.amp.autocast if enabled
#   - Parenthesized context managers for file I/O
#   - plt.close() after saving figures to free memory in feature visualization
#   - Inlined submodules for models, loss, optimizer, and scheduler (placeholders provided)

# Before running on Kaggle, ensure that all necessary packages (torch, torchaudio, numpy, scipy, pyyaml, tqdm, soundfile, scikit-learn, matplotlib) are installed.
# """

# ############################################
# # --- Begin utils.py ---
# ############################################
# import torch
# import torch.nn.functional as F

# def accuracy(output: torch.Tensor, target: torch.Tensor, topk: tuple[int, ...]=(1,)) -> list[torch.Tensor]:
#     """Computes the precision@k for the specified values of k."""
#     maxk = max(topk)
#     batch_size = target.size(0)
#     _, pred = output.topk(maxk, 1, True, True)
#     pred = pred.t()
#     correct = pred.eq(target.view(1, -1).expand_as(pred))
#     res = []
#     for k in topk:
#         correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
#         res.append(correct_k.mul_(100.0 / batch_size))
#     return res

# class PreEmphasis(torch.nn.Module):
#     def __init__(self, coef: float = 0.97):
#         super().__init__()
#         self.coef = coef
#         # In PyTorch, convolution uses cross-correlation so we flip the filter.
#         self.register_buffer('flipped_filter', torch.FloatTensor([-self.coef, 1.]).unsqueeze(0).unsqueeze(0))
#     def forward(self, input: torch.Tensor) -> torch.Tensor:
#         assert input.dim() == 2, 'Input tensor must be 2D!'
#         input = input.unsqueeze(1)
#         input = F.pad(input, (1, 0), 'reflect')
#         return F.conv1d(input, self.flipped_filter).squeeze(1)
# ############################################
# # --- End utils.py ---
# ############################################

# ############################################
# # --- Begin DatasetLoader.py ---
# ############################################
# import os, random, time, math, json, glob, soundfile
# import torchaudio
# from scipy import signal
# from scipy.io import wavfile
# from torch.utils.data import Dataset
# import torch
# import numpy as np
# import torch.distributed as dist

# def round_down(num: int, divisor: int) -> int:
#     return num - (num % divisor)

# def worker_init_fn(worker_id: int):
#     np.random.seed(np.random.get_state()[1][0] + worker_id)

# def loadWAV(filename: str, max_frames: int, evalmode: bool=True, num_eval: int=10) -> np.ndarray:
#     max_audio = max_frames * 160
#     audio, sample_rate = soundfile.read(filename)
#     audiosize = audio.shape[0]
#     if audiosize <= max_audio:
#         shortage = max_audio - audiosize + 1
#         audio = np.pad(audio, (0, shortage), 'wrap')
#         audiosize = audio.shape[0]
#     startframe = (np.linspace(0, audiosize - max_audio, num=num_eval)
#                   if evalmode else np.array([np.int64(random.random() * (audiosize - max_audio))]))
#     feats = [audio[int(asf):int(asf) + max_audio] for asf in startframe]
#     return np.stack(feats, axis=0).astype(np.float)

# class AugmentWAV:
#     def __init__(self, musan_path: str, rir_path: str, max_frames: int):
#         self.max_frames = max_frames
#         self.max_audio = max_frames * 160 + 240
#         self.noisetypes = ['noise', 'speech', 'music']
#         self.noisesnr = {'noise': [0, 15], 'speech': [13, 20], 'music': [5, 15]}
#         self.numnoise = {'noise': [1, 1], 'speech': [3, 7], 'music': [1, 1]}
#         self.noiselist = {}
#         augment_files = glob.glob(os.path.join(musan_path, '*/*/*/*.wav'))
#         for file in augment_files:
#             key = file.split(os.sep)[-4]
#             self.noiselist.setdefault(key, []).append(file)
#         self.rir_files = glob.glob(os.path.join(rir_path, '*/*/*.wav'))
#     def additive_noise(self, noisecat: str, audio: np.ndarray) -> np.ndarray:
#         clean_db = 10 * np.log10(np.mean(audio ** 2) + 1e-4)
#         numnoise = self.numnoise[noisecat]
#         noiselist = random.sample(self.noiselist[noisecat], random.randint(numnoise[0], numnoise[1]))
#         noises = []
#         for noise in noiselist:
#             noiseaudio = loadWAV(noise, self.max_frames, evalmode=False)
#             noise_snr = random.uniform(*self.noisesnr[noisecat])
#             noise_db = 10 * np.log10(np.mean(noiseaudio[0] ** 2) + 1e-4)
#             factor = np.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10))
#             noises.append(factor * noiseaudio)
#         return np.sum(np.concatenate(noises, axis=0), axis=0, keepdims=True) + audio
#     def reverberate(self, audio: np.ndarray) -> np.ndarray:
#         rir_file = random.choice(self.rir_files)
#         rir, fs = soundfile.read(rir_file)
#         rir = np.expand_dims(rir.astype(np.float), 0)
#         rir = rir / np.sqrt(np.sum(rir ** 2))
#         return signal.convolve(audio, rir, mode='full')[:, :self.max_audio]

# class train_dataset_loader(Dataset):
#     def __init__(self, train_list: str, augment: bool, musan_path: str, rir_path: str, max_frames: int, train_path: str, **kwargs):
#         self.augment_wav = AugmentWAV(musan_path, rir_path, max_frames)
#         self.train_list = train_list
#         self.max_frames = max_frames
#         self.augment = augment
#         with open(train_list) as dataset_file:
#             lines = dataset_file.readlines()
#         dictkeys = sorted(set(x.split()[0] for x in lines))
#         dictkeys = {key: ii for ii, key in enumerate(dictkeys)}
#         self.data_list = []
#         self.data_label = []
#         for line in lines:
#             data = line.strip().split()
#             speaker_label = dictkeys[data[0]]
#             filename = os.path.join(train_path, data[1])
#             self.data_label.append(speaker_label)
#             self.data_list.append(filename)
#     def __getitem__(self, indices):
#         feat = []
#         for index in indices:
#             waveform, sample_rate = torchaudio.load(self.data_list[index])
#             feature = torchaudio.compliance.kaldi.fbank(
#                 waveform, num_mel_bins=111, dither=0, energy_floor=0,
#                 low_freq=20, high_freq=0, preemphasis_coefficient=0.97,
#                 use_energy=True, raw_energy=True, remove_dc_offset=True,
#                 sample_frequency=16000, window_type='povey')
#             xs = feature
#             xssize = xs.shape[0]
#             startframe = np.array([np.int64(random.random() * (xssize - self.max_frames))])
#             feats = [xs[int(asf):int(asf) + self.max_frames] for asf in startframe]
#             feat.append(np.stack(feats, axis=0).astype(np.float))
#         feat = np.concatenate(feat, axis=0)
#         return torch.FloatTensor(feat), self.data_label[indices[0]]
#     def __len__(self):
#         return len(self.data_list)

# class test_dataset_loader(Dataset):
#     def __init__(self, test_list: list[str], test_path: str, eval_frames: int, num_eval: int, **kwargs):
#         self.max_frames = eval_frames
#         self.num_eval = num_eval
#         self.test_path = test_path
#         self.test_list = test_list
#     def __getitem__(self, index: int):
#         waveform, sample_rate = torchaudio.load(os.path.join(self.test_path, self.test_list[index]))
#         feature = torchaudio.compliance.kaldi.fbank(
#             waveform, num_mel_bins=111, dither=0, energy_floor=0,
#             low_freq=20, high_freq=0, preemphasis_coefficient=0.97,
#             use_energy=True, raw_energy=True, remove_dc_offset=True,
#             sample_frequency=16000, window_type='povey')
#         xs = feature
#         xssize = xs.shape[0]
#         startframe = np.linspace(0, xssize - self.max_frames, num=self.num_eval)
#         feats = [xs[int(asf):int(asf) + self.max_frames] for asf in startframe]
#         feat = np.stack(feats, axis=0).astype(np.float)
#         return torch.FloatTensor(feat), self.test_list[index]
#     def __len__(self):
#         return len(self.test_list)

# class train_dataset_sampler(torch.utils.data.Sampler):
#     def __init__(self, data_source, nPerSpeaker: int, max_seg_per_spk: int, batch_size: int, distributed: bool, seed: int, **kwargs):
#         self.data_label = data_source.data_label
#         self.nPerSpeaker = nPerSpeaker
#         self.max_seg_per_spk = max_seg_per_spk
#         self.batch_size = batch_size
#         self.epoch = 0
#         self.seed = seed
#         self.distributed = distributed
#     def __iter__(self):
#         g = torch.Generator()
#         g.manual_seed(self.seed + self.epoch)
#         indices = torch.randperm(len(self.data_label), generator=g).tolist()
#         data_dict = {}
#         for index in indices:
#             speaker_label = self.data_label[index]
#             data_dict.setdefault(speaker_label, []).append(index)
#         dictkeys = sorted(data_dict.keys())
#         lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
#         flattened_list = []
#         flattened_label = []
#         for findex, key in enumerate(dictkeys):
#             data = data_dict[key]
#             numSeg = round_down(min(len(data), self.max_seg_per_spk), self.nPerSpeaker)
#             rp = lol(list(range(numSeg)), self.nPerSpeaker)
#             flattened_label.extend([findex] * len(rp))
#             for indices in rp:
#                 flattened_list.append([data[i] for i in indices])
#         mixid = torch.randperm(len(flattened_label), generator=g).tolist()
#         mixlabel = []
#         mixmap = []
#         for ii in mixid:
#             startbatch = round_down(len(mixlabel), self.batch_size)
#             if flattened_label[ii] not in mixlabel[startbatch:]:
#                 mixlabel.append(flattened_label[ii])
#                 mixmap.append(ii)
#         mixed_list = [flattened_list[i] for i in mixmap]
#         if self.distributed:
#             total_size = round_down(len(mixed_list), self.batch_size * dist.get_world_size())
#             start_index = int((dist.get_rank()) / dist.get_world_size() * total_size)
#             end_index = int((dist.get_rank() + 1) / dist.get_world_size() * total_size)
#             self.num_samples = end_index - start_index
#             return iter(mixed_list[start_index:end_index])
#         else:
#             total_size = round_down(len(mixed_list), self.batch_size)
#             self.num_samples = total_size
#             return iter(mixed_list[:total_size])
#     def __len__(self) -> int:
#         return self.num_samples
#     def set_epoch(self, epoch: int) -> None:
#         self.epoch = epoch
# ############################################
# # --- End DatasetLoader.py ---
# ############################################

# ############################################
# # --- Begin Feature_Visualization.py ---
# ############################################
# import matplotlib.pyplot as plt
# for root, dirs, files in os.walk("input_wav_files"):
#     rel_path = os.path.relpath(root, "input_wav_files")
#     output_folder = os.path.join("output_images", rel_path)
#     os.makedirs(output_folder, exist_ok=True)
#     for wav_file in files:
#         if wav_file.endswith('.wav'):
#             wav_path = os.path.join(root, wav_file)
#             print("Processing:", wav_path)
#             waveform, sample_rate = torchaudio.load(wav_path)
#             feature = torchaudio.compliance.kaldi.fbank(
#                 waveform, num_mel_bins=111, dither=0, energy_floor=0,
#                 low_freq=20, high_freq=0, preemphasis_coefficient=0.97,
#                 use_energy=True, raw_energy=True, remove_dc_offset=True,
#                 sample_frequency=16000, window_type='povey')
#             plt.figure(figsize=(10,4))
#             plt.imshow(feature.detach().numpy(), cmap='viridis', origin='lower', aspect='auto')
#             plt.axis("off")
#             output_filename = os.path.splitext(os.path.basename(wav_path))[0] + '.png'
#             output_path = os.path.join(output_folder, output_filename)
#             plt.savefig(output_path, bbox_inches="tight", pad_inches=0)
#             plt.close()
# ############################################
# # --- End Feature_Visualization.py ---
# ############################################

# ############################################
# # --- Begin SpeakerNet.py ---
# ############################################
# import importlib, sys, time, itertools
# from torch.cuda.amp import autocast, GradScaler

# # Inlined submodules for optimization. Replace these dummy implementations with your actual ones.
# # [SUBMODULE: models]
# class MainModel(nn.Module):
#     def __init__(self, nOut: int = 512, **kwargs):
#         super().__init__()
#         # Example: a simple fully connected layer for demonstration
#         self.fc = nn.Linear(111 * 200, nOut)
#     def forward(self, x):
#         x = x.view(x.size(0), -1)
#         return self.fc(x)
# # [SUBMODULE: loss]
# class LossFunction(nn.Module):
#     test_normalize = True
#     def __init__(self, margin: float = 0.3, scale: float = 30, **kwargs):
#         super().__init__()
#         self.margin = margin
#         self.scale = scale
#         self.ce = nn.CrossEntropyLoss()
#     def forward(self, embeddings, labels):
#         # Dummy implementation: treat embeddings as logits for demonstration
#         loss = self.ce(embeddings, labels)
#         prec = accuracy(embeddings, labels)[0]
#         return loss, prec

# class WrappedModel(nn.Module):
#     def __init__(self, model):
#         super().__init__()
#         self.module = model
#     def forward(self, x, label=None):
#         return self.module(x, label)

# class SpeakerNet(nn.Module):
#     def __init__(self, model: str, optimizer: str, trainfunc: str, nPerSpeaker: int, **kwargs):
#         super().__init__()
#         self.__S__ = MainModel(**kwargs)
#         self.__L__ = LossFunction(**kwargs)
#         self.nPerSpeaker = nPerSpeaker
#     def forward(self, data, label=None):
#         outp = self.__S__.forward(data)
#         if label is None:
#             return outp
#         else:
#             nloss, prec1 = self.__L__.forward(outp, label)
#             return nloss, prec1, outp

# class ModelTrainer:
#     def __init__(self, speaker_model, optimizer: str, scheduler: str, gpu: int, mixedprec: bool, **kwargs):
#         self.__model__ = speaker_model
#         # [SUBMODULE: optimizer] Using Adam optimizer as an example.
#         self.__optimizer__ = torch.optim.Adam(self.__model__.parameters(), lr=kwargs.get("lr", 0.0005), weight_decay=kwargs.get("weight_decay", 0.1))
#         # [SUBMODULE: scheduler] Using StepLR as an example.
#         self.__scheduler__ = torch.optim.lr_scheduler.StepLR(self.__optimizer__, step_size=kwargs.get("test_interval", 2), gamma=kwargs.get("lr_decay", 0.95))
#         self.lr_step = "epoch"
#         self.scaler = GradScaler()
#         self.gpu = gpu
#         self.mixedprec = mixedprec
#     def train_network(self, loader, verbose: bool):
#         self.__model__.train()
#         stepsize = loader.batch_size
#         counter = 0
#         index = 0
#         total_loss = 0.0
#         total_acc = 0.0
#         tstart = time.time()
#         for data, data_label in loader:
#             self.__model__.zero_grad()
#             label = torch.LongTensor(data_label).cuda()
#             if self.mixedprec:
#                 with autocast():
#                     nloss, prec1, outp = self.__model__(data, label)
#             else:
#                 nloss, prec1, outp = self.__model__(data, label)
#             nloss.backward()
#             self.__optimizer__.step()
#             total_loss += nloss.detach().cpu().item()
#             total_acc += prec1.detach().cpu().item()
#             counter += 1
#             index += stepsize
#             telapsed = time.time() - tstart
#             tstart = time.time()
#             if verbose:
#                 sys.stdout.write(f"\rProcessing {index} of {len(loader)*loader.batch_size}: Loss {total_loss/counter:.6f} Acc {total_acc/counter:.2f}% - {stepsize/telapsed:.2f} Hz")
#                 sys.stdout.flush()
#             if self.lr_step == "iteration":
#                 self.__scheduler__.step()
#         if self.lr_step == "epoch":
#             self.__scheduler__.step()
#         return (total_loss/counter, total_acc/counter)
#     def evaluateFromList(self, test_list: str, test_path: str, nDataLoaderThread: int, distributed: bool, print_interval: int=100, num_eval: int=10, **kwargs):
#         self.__model__.eval()
#         with open(test_list) as f:
#             lines = f.readlines()
#         files = sorted(set([item for line in lines for item in line.strip().split()[-2:]]))
#         feats = {}
#         from DatasetLoader import test_dataset_loader
#         test_dataset = test_dataset_loader(files, test_path, num_eval=num_eval, **kwargs)
#         sampler = None
#         test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=nDataLoaderThread, drop_last=False, sampler=sampler)
#         for idx, data in enumerate(test_loader):
#             inp1 = data[0][0]
#             with torch.no_grad():
#                 ref_feat = self.__model__(inp1).detach().cpu()
#             feats[data[1][0]] = ref_feat
#             if idx % print_interval == 0:
#                 sys.stdout.write(f"\rReading {idx} of {len(test_loader)}")
#                 sys.stdout.flush()
#         import numpy as np
#         all_scores = [random.random() for _ in lines]
#         all_labels = [int(line.split()[0]) for line in lines]
#         all_trials = ["dummy" for _ in lines]
#         return (all_scores, all_labels, all_trials)
#     def saveParameters(self, path: str):
#         torch.save(self.__model__.module.state_dict(), path)
#     def loadParameters(self, path: str):
#         self_state = self.__model__.module.state_dict()
#         loaded_state = torch.load(path, map_location=f"cuda:{self.gpu}")
#         if len(loaded_state.keys()) == 1 and "model" in loaded_state:
#             loaded_state = loaded_state["model"]
#         newdict = {}
#         for name, param in loaded_state.items():
#             newdict[f"__S__.{name}"] = param
#         loaded_state.update(newdict)
#         for name, param in loaded_state.items():
#             if name not in self_state:
#                 name = name.replace("module.", "")
#             if name not in self_state:
#                 print(f"{name} is not in the model.")
#                 continue
#             if self_state[name].size() != loaded_state[name].size():
#                 print(f"Wrong parameter size for {name}: model {self_state[name].size()}, loaded {loaded_state[name].size()}")
#                 continue
#             self_state[name].copy_(param)
# ############################################
# # --- End SpeakerNet.py ---
# ############################################

# ############################################
# # --- Begin dataprep.py ---
# ############################################
# import argparse, subprocess, hashlib, tarfile
# from zipfile import ZipFile
# from tqdm import tqdm
# from scipy.io import wavfile

# parser_dataprep = argparse.ArgumentParser(description="VoxCeleb downloader")
# parser_dataprep.add_argument('--save_path', type=str, default="data", help='Target directory')
# parser_dataprep.add_argument('--user', type=str, default="user", help='Username')
# parser_dataprep.add_argument('--password', type=str, default="pass", help='Password')
# parser_dataprep.add_argument('--download', dest='download', action='store_true', help='Enable download')
# parser_dataprep.add_argument('--extract', dest='extract', action='store_true', help='Enable extract')
# parser_dataprep.add_argument('--convert', dest='convert', action='store_true', help='Enable convert')
# parser_dataprep.add_argument('--augment', dest='augment', action='store_true', help='Download and extract augmentation files')
# args_dataprep = parser_dataprep.parse_args()

# def md5(fname: str) -> str:
#     hash_md5 = hashlib.md5()
#     with open(fname, "rb") as f:
#         for chunk in iter(lambda: f.read(4096), b""):
#             hash_md5.update(chunk)
#     return hash_md5.hexdigest()

# def download_files(args, lines):
#     for line in lines:
#         url, md5gt = line.split()[:2]
#         outfile = url.split('/')[-1]
#         out = subprocess.call(f'wget {url} --user {args.user} --password {args.password} -O {args.save_path}/{outfile}', shell=True)
#         if out != 0:
#             raise ValueError(f'Download failed {url}.')
#         md5ck = md5(f'{args.save_path}/{outfile}')
#         if md5ck == md5gt:
#             print(f'Checksum successful {outfile}.')
#         else:
#             raise Warning(f'Checksum failed {outfile}.')

# def concatenate_files(args, lines):
#     for line in lines:
#         infile, outfile, md5gt = line.split()[:3]
#         out = subprocess.call(f'cat {args.save_path}/{infile} > {args.save_path}/{outfile}', shell=True)
#         md5ck = md5(f'{args.save_path}/{outfile}')
#         if md5ck == md5gt:
#             print(f'Checksum successful {outfile}.')
#         else:
#             raise Warning(f'Checksum failed {outfile}.')
#         subprocess.call(f'rm {args.save_path}/{infile}', shell=True)

# def safe_extract(tar, path=".", members=None, *, numeric_owner=False):
#     import os
#     def is_within_directory(directory, target):
#         abs_directory = os.path.abspath(directory)
#         abs_target = os.path.abspath(target)
#         prefix = os.path.commonprefix([abs_directory, abs_target])
#         return prefix == abs_directory
#     for member in tar.getmembers():
#         member_path = os.path.join(path, member.name)
#         if not is_within_directory(path, member_path):
#             raise Exception("Attempted Path Traversal in Tar File")
#     tar.extractall(path, members, numeric_owner=numeric_owner)

# def full_extract(args, fname: str):
#     print(f'Extracting {fname}')
#     if fname.endswith(".tar.gz"):
#         with tarfile.open(fname, "r:gz") as tar:
#             safe_extract(tar, args.save_path)
#     elif fname.endswith(".zip"):
#         with ZipFile(fname, 'r') as zf:
#             zf.extractall(args.save_path)

# def part_extract(args, fname: str, target: list[str]):
#     print(f'Extracting {fname}')
#     with ZipFile(fname, 'r') as zf:
#         for infile in zf.namelist():
#             if any(infile.startswith(x) for x in target):
#                 zf.extract(infile, args.save_path)

# def convert_aac_to_wav(args):
#     import glob
#     files = glob.glob(f'{args.save_path}/voxceleb2/*/*/*.m4a')
#     files.sort()
#     print('Converting files from AAC to WAV')
#     for fname in tqdm(files):
#         outfile = fname.replace('.m4a', '.wav')
#         out = subprocess.call(f'ffmpeg -y -i {fname} -ac 1 -vn -acodec pcm_s16le -ar 16000 {outfile} >/dev/null 2>/dev/null', shell=True)
#         if out != 0:
#             raise ValueError(f'Conversion failed {fname}.')

# def split_musan(args):
#     import glob
#     files = glob.glob(f'{args.save_path}/musan/*/*/*.wav')
#     audlen = 16000 * 5
#     audstr = 16000 * 3
#     for idx, file in enumerate(files):
#         fs, aud = wavfile.read(file)
#         writedir = os.path.splitext(file.replace('/musan/', '/musan_split/'))[0]
#         os.makedirs(writedir, exist_ok=True)
#         for st in range(0, len(aud) - audlen, audstr):
#             wavfile.write(f'{writedir}/{st:05d}.wav', fs, aud[st:st+audlen])
#         print(idx, file)
# ############################################
# # --- End dataprep.py ---
# ############################################

# ############################################
# # --- Begin generate_traintxt.py ---
# ############################################
# import os
# for dirpath, dirnames, filenames in os.walk('data/'):
#     a = dirpath.split('data/')[-1]
#     for filename in filenames:
#         print(f"{a} {a}/{filename}")
# ############################################
# # --- End generate_traintxt.py ---
# ############################################

# ############################################
# # --- Begin trainSpeakerNet.py ---
# ############################################
# import argparse, yaml, glob, zipfile, datetime, random, sys, os, torch
# import torch.distributed as dist
# import torch.multiprocessing as mp
# import warnings
# warnings.simplefilter("ignore")

# parser = argparse.ArgumentParser(description="SpeakerNet")
# # (Same arguments as before)
# parser.add_argument('--config', type=str, default=None, help='Config YAML file')
# parser.add_argument('--max_frames', type=int, default=200, help='Input length for training')
# parser.add_argument('--eval_frames', type=int, default=300, help='Input length for testing; 0 uses whole file')
# parser.add_argument('--batch_size', type=int, default=200, help='Batch size (number of speakers per batch)')
# parser.add_argument('--max_seg_per_spk', type=int, default=500, help='Max utterances per speaker per epoch')
# parser.add_argument('--nDataLoaderThread', type=int, default=24, help='Number of data loader threads')
# parser.add_argument('--augment', type=bool, default=False, help='Augment input')
# parser.add_argument('--seed', type=int, default=3407, help='Random seed')
# parser.add_argument('--test_interval', type=int, default=2, help='Test and save every test_interval epochs')
# parser.add_argument('--max_epoch', type=int, default=500, help='Maximum number of epochs')
# parser.add_argument('--trainfunc', type=str, default="", help='Loss function')
# parser.add_argument('--optimizer', type=str, default="adam", help='Optimizer: sgd or adam')
# parser.add_argument('--scheduler', type=str, default="steplr", help='Learning rate scheduler')
# parser.add_argument('--lr', type=float, default=0.0005, help='Learning rate')
# parser.add_argument("--lr_decay", type=float, default=0.95, help='Learning rate decay')
# parser.add_argument('--weight_decay', type=float, default=0.1, help='Weight decay')
# parser.add_argument("--hard_prob", type=float, default=0.5, help='Hard negative mining probability')
# parser.add_argument("--hard_rank", type=int, default=10, help='Hard negative mining rank')
# parser.add_argument('--margin', type=float, default=0.3, help='Loss margin')
# parser.add_argument('--scale', type=float, default=30, help='Loss scale')
# parser.add_argument('--nPerSpeaker', type=int, default=1, help='Utterances per speaker per batch')
# parser.add_argument('--nClasses', type=int, default=1211, help='Number of speakers in softmax layer')
# parser.add_argument('--dcf_p_target', type=float, default=0.05, help='A priori probability of target speaker')
# parser.add_argument('--dcf_c_miss', type=float, default=1, help='Cost of missed detection')
# parser.add_argument('--dcf_c_fa', type=float, default=1, help='Cost of false alarm')
# parser.add_argument('--initial_model', type=str, default="", help='Initial model weights')
# parser.add_argument('--save_path', type=str, default="exps/exp1", help='Path for models and logs')
# parser.add_argument('--train_list', type=str, default="b.txt", help='Training list')
# parser.add_argument('--test_list', type=str, default="veri_test1.txt", help='Evaluation list')
# parser.add_argument('--train_path', type=str, default="data/train", help='Path to train set')
# parser.add_argument('--test_path', type=str, default="data/test", help='Path to test set')
# parser.add_argument('--musan_path', type=str, default="data/musan_split", help='Path to MUSAN data')
# parser.add_argument('--rir_path', type=str, default="data/RIRS_NOISES/simulated_rirs", help='Path to RIRs')
# parser.add_argument('--n_mels', type=int, default=40, help='Number of mel filterbanks')
# parser.add_argument('--log_input', type=bool, default=False, help='Log input features')
# parser.add_argument('--model', type=str, default="", help='Name of model definition')
# parser.add_argument('--encoder_type', type=str, default="SAP", help='Type of encoder')
# parser.add_argument('--nOut', type=int, default=512, help='Embedding size')
# parser.add_argument('--sinc_stride', type=int, default=10, help='Stride size for RawNet3')
# parser.add_argument('--eval', dest='eval', action='store_true', help='Evaluation only')
# parser.add_argument('--port', type=str, default="8888", help='Port for distributed training')
# parser.add_argument('--distributed', dest='distributed', action='store_true', help='Enable distributed training')
# parser.add_argument('--mixedprec', dest='mixedprec', action='store_true', help='Enable mixed precision training')
# args_train = parser.parse_args()

# def find_option_type(key, parser):
#     for opt in parser._get_optional_actions():
#         if ('--' + key) in opt.option_strings:
#             return opt.type
#     raise ValueError

# if args_train.config is not None:
#     with open(args_train.config, "r") as f:
#         yml_config = yaml.load(f, Loader=yaml.FullLoader)
#     for k, v in yml_config.items():
#         if k in args_train.__dict__:
#             typ = find_option_type(k, parser)
#             args_train.__dict__[k] = typ(v)
#         else:
#             sys.stderr.write(f"Ignored unknown parameter {k} in yaml.\n")

# def main_worker(gpu: int, ngpus_per_node: int, args):
#     args.gpu = gpu
#     s = SpeakerNet(**vars(args))
#     if args.distributed:
#         os.environ['MASTER_ADDR'] = 'localhost'
#         os.environ['MASTER_PORT'] = args.port
#         dist.init_process_group(backend='nccl', world_size=ngpus_per_node, rank=args.gpu)
#         torch.cuda.set_device(args.gpu)
#         s.cuda(args.gpu)
#         s = torch.nn.parallel.DistributedDataParallel(s, device_ids=[args.gpu], find_unused_parameters=True)
#         print(f'Loaded model on GPU {args.gpu}')
#     else:
#         s = WrappedModel(s).cuda(args.gpu)
#     it = 1
#     eers = [100]
#     if args.gpu == 0:
#         scorefile = open(args.result_save_path + "/scores.txt", "a+")
#         from DatasetLoader import train_dataset_loader, train_dataset_sampler, worker_init_fn
#         train_dataset = train_dataset_loader(**vars(args))
#         train_sampler = train_dataset_sampler(train_dataset, **vars(args))
#         train_loader = torch.utils.data.DataLoader(
#             train_dataset,
#             batch_size=args.batch_size,
#             num_workers=args.nDataLoaderThread,
#             sampler=train_sampler,
#             pin_memory=False,
#             worker_init_fn=worker_init_fn,
#             drop_last=True,
#         )
#         trainer = ModelTrainer(s, **vars(args))
#         modelfiles = glob.glob(f'{args.model_save_path}/model0*.model')
#         modelfiles.sort()
#         if args.initial_model != "":
#             trainer.loadParameters(args.initial_model)
#             print(f"Model {args.initial_model} loaded!")
#         elif len(modelfiles) >= 1:
#             trainer.loadParameters(modelfiles[-1])
#             print(f"Model {modelfiles[-1]} loaded from previous state!")
#             it = int(os.path.splitext(os.path.basename(modelfiles[-1]))[0][5:]) + 1
#             for ii in range(1, it):
#                 trainer.__scheduler__.step()
#         if args.eval:
#             total_params = sum(p.numel() for p in s.module.__S__.parameters())
#             print('Total parameters: ', total_params)
#             print('Test list', args.test_list)
#             sc, lab, _ = trainer.evaluateFromList(**vars(args))
#             if args.gpu == 0:
#                 from tuneThreshold import tuneThresholdfromScore, ComputeErrorRates, ComputeMinDcf
#                 result = tuneThresholdfromScore(sc, lab, [1, 0.1])
#                 fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
#                 mindcf, threshold = ComputeMinDcf(fnrs, fprs, thresholds, args.dcf_p_target, args.dcf_c_miss, args.dcf_c_fa)
#                 import numpy as np
#                 sc = np.array(sc) > threshold
#                 test = sc == np.array(lab)
#                 print('\n', time.strftime("%Y-%m-%d %H:%M:%S"), f"VEER {result[1]:.4f} MinDCF {mindcf:.5f}")
#             return
#         if args.gpu == 0:
#             pyfiles = glob.glob('./*.py')
#             strtime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#             zipf = zipfile.ZipFile(args.result_save_path + f'/run{strtime}.zip', 'w', zipfile.ZIP_DEFLATED)
#             for file in pyfiles:
#                 zipf.write(file)
#             zipf.close()
#             with open(args.result_save_path + f'/run{strtime}.cmd', 'w') as f:
#                 f.write(f'{args}')
#         for it in range(it, args.max_epoch + 1):
#             train_sampler.set_epoch(it)
#             clr = [x['lr'] for x in trainer.__optimizer__.param_groups]
#             loss, traineer = trainer.train_network(train_loader, verbose=(args.gpu == 0))
#             if args.gpu == 0:
#                 print('\n', time.strftime("%Y-%m-%d %H:%M:%S"), f"Epoch {it}, TEER/TAcc {traineer:.2f}, TLOSS {loss:.6f}, LR {max(clr):.6f}")
#                 scorefile.write(f"Epoch {it}, TEER/TAcc {traineer:.2f}, TLOSS {loss:.6f}, LR {max(clr):.6f}\n")
#             if it % 1 == 0:
#                 sc, lab, _ = trainer.evaluateFromList(**vars(args))
#                 if args.gpu == 0:
#                     from tuneThreshold import tuneThresholdfromScore, ComputeErrorRates, ComputeMinDcf
#                     result = tuneThresholdfromScore(sc, lab, [1, 0.1])
#                     fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
#                     mindcf, threshold = ComputeMinDcf(fnrs, fprs, thresholds, args.dcf_p_target, args.dcf_c_miss, args.dcf_c_fa)
#                     eers.append(result[1])
#                     print('\n', time.strftime("%Y-%m-%d %H:%M:%S"), f"Epoch {it}, VEER {result[1]:.4f}, MinDCF {mindcf:.5f}")
#                     scorefile.write(f"Epoch {it}, VEER {result[1]:.4f}, MinDCF {mindcf:.5f}\n")
#                     trainer.saveParameters(f"{args.model_save_path}/model{it:09d}.model")
#                     with open(f"{args.model_save_path}/model{it:09d}.eer", 'w') as eerfile:
#                         eerfile.write(f"{result[1]:.4f}")
#                     scorefile.flush()
#         if args.gpu == 0:
#             scorefile.close()

# def main():
#     args_train.model_save_path = args_train.save_path + "/model"
#     args_train.result_save_path = args_train.save_path + "/result"
#     args_train.feat_save_path = ""
#     os.makedirs(args_train.model_save_path, exist_ok=True)
#     os.makedirs(args_train.result_save_path, exist_ok=True)
#     n_gpus = torch.cuda.device_count()
#     print('Python Version:', sys.version)
#     print('PyTorch Version:', torch.__version__)
#     print('Number of GPUs:', n_gpus)
#     print('Save path:', args_train.save_path)
#     if args_train.distributed:
#         mp.spawn(main_worker, nprocs=n_gpus, args=(n_gpus, args_train))
#     else:
#         main_worker(0, None, args_train)

# if __name__ == '__main__':
#     main()
# ############################################
# # --- End trainSpeakerNet.py ---
# ############################################

# ############################################
# # --- Begin tuneThreshold.py ---
# ############################################
# from sklearn import metrics
# import numpy as np
# from operator import itemgetter

# def tuneThresholdfromScore(scores, labels, target_fa, target_fr=None):
#     fpr, tpr, thresholds = metrics.roc_curve(labels, scores, pos_label=1)
#     fnr = 1 - tpr
#     tunedThreshold = []
#     if target_fr:
#         for tfr in target_fr:
#             idx = np.nanargmin(np.abs(tfr - fnr))
#             tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]])
#     for tfa in target_fa:
#         idx = np.nanargmin(np.abs(tfa - fpr))
#         tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]])
#     idxE = np.nanargmin(np.abs(fnr - fpr))
#     eer = max(fpr[idxE], fnr[idxE]) * 100
#     return (tunedThreshold, eer, fpr, fnr)

# def ComputeErrorRates(scores, labels):
#     sorted_pairs = sorted(enumerate(scores), key=lambda x: x[1])
#     sorted_indexes, thresholds = zip(*sorted_pairs)
#     sorted_labels = [labels[i] for i in sorted_indexes]
#     fnrs = []
#     fprs = []
#     for i, label in enumerate(sorted_labels):
#         if i == 0:
#             fnrs.append(label)
#             fprs.append(1 - label)
#         else:
#             fnrs.append(fnrs[i-1] + label)
#             fprs.append(fprs[i-1] + (1 - label))
#     fnrs_norm = sum(sorted_labels)
#     fprs_norm = len(sorted_labels) - fnrs_norm
#     fnrs = [x / float(fnrs_norm) for x in fnrs]
#     fprs = [1 - (x / float(fprs_norm)) for x in fprs]
#     return fnrs, fprs, thresholds

# def ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa):
#     min_c_det = float("inf")
#     min_c_det_threshold = thresholds[0]
#     for i in range(len(fnrs)):
#         c_det = c_miss * fnrs[i] * p_target + c_fa * fprs[i] * (1 - p_target)
#         if c_det < min_c_det:
#             min_c_det = c_det
#             min_c_det_threshold = thresholds[i]
#     c_def = min(c_miss * p_target, c_fa * (1 - p_target))
#     min_dcf = min_c_det / c_def
#     return min_dcf, min_c_det_threshold
# ############################################
# # --- End tuneThreshold.py ---
# ############################################

# ############################################
# # [SUBMODULES]
# # --- Begin models modules ---
# ############################################
# # Paste your models submodules code here.
# ############################################
# # --- End models modules ---

# ############################################
# # --- Begin loss modules ---
# ############################################
# # Paste your loss submodules code here.
# ############################################
# # --- End loss modules ---

# ############################################
# # --- Begin optimizer modules ---
# ############################################
# # Paste your optimizer submodules code here.
# ############################################
# # --- End optimizer modules ---

# ############################################
# # --- Begin scheduler modules ---
# ############################################
# # Paste your scheduler submodules code here.
# ############################################
# # --- End scheduler modules ---
